## Regressão Logística

De volta pro assunto de classificação, o objetivo desse conteúdo é explorar a representação de texto em computadores e utilizar regressão logística para categorizar os documentos. Além disso, tem um desafio no final para implementar otimização através de algoritmos genéticos.

In [1]:
import pandas as pd
import numpy  as np

df = pd.read_csv('movie_reviews.tsv', sep='\t', index_col=None)

df.columns

Index(['id', 'sentiment', 'review'], dtype='object')

In [2]:
X = df.review
y = df.sentiment

In [3]:
X.shape

(413,)

In [4]:
print(X[0])

With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 min

In [5]:
from collections import Counter

# reune todas as palavras/tokens das amostras
palavras = []
for s in X.values:
    palavras += s.lower().split()

dicionario = Counter(palavras)
print(dicionario.most_common(20))

[('the', 5108), ('a', 2621), ('and', 2515), ('of', 2366), ('to', 2150), ('is', 1748), ('in', 1470), ('this', 1133), ('i', 1125), ('that', 1070), ('it', 1036), ('with', 764), ('for', 718), ('as', 708), ('but', 685), ('was', 676), ('on', 529), ('movie', 501), ('film', 492), ('are', 473)]


In [6]:
# separa qual sera o vocabulario
vocab = []
for palavra, contagem in dicionario.most_common(100):
    vocab.append(palavra)

print(vocab)

['the', 'a', 'and', 'of', 'to', 'is', 'in', 'this', 'i', 'that', 'it', 'with', 'for', 'as', 'but', 'was', 'on', 'movie', 'film', 'are', 'his', 'not', 'be', 'have', 'you', 'one', 'by', 'at', 'he', 'all', 'an', 'from', 'who', 'so', 'her', 'like', 'some', 'has', 'about', 'just', "it's", 'or', 'out', 'if', 'they', 'there', 'very', 'when', 'their', 'more', 'good', 'only', 'would', 'what', 'even', 'no', 'really', 'up', 'my', 'which', 'she', 'how', 'had', '-', 'can', 'into', 'will', 'than', 'see', 'get', 'other', 'been', 'were', 'make', 'because', 'its', 'people', 'much', 'first', 'most', 'we', 'also', 'me', 'could', "don't", 'story', 'bad', 'time', 'being', 'then', 'do', 'great', 'any', 'after', 'many', 'made', 'well', '&', 'think', 'way']


In [7]:
# cria a amostra marcando quais palavras aparecem naquela sentença (bag of words)

X_bow = []
for _ in range(df.review.shape[0]):
    s = df.review.values[0]
    sample = []
    for p in vocab:
        if p in s:
            sample.append(1.0)
        else:
            sample.append(0.0)
    X_bow.append(sample)

X_bow = np.array(X_bow)

In [8]:
# separa em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_bow, y, test_size=0.33, random_state=42)

In [9]:
# regressão logística e funções auxiliares

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def loss(model, X, y):
    X = add_bias(X)
    z = np.dot(X, model)
    h = sigmoid(z)
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

def add_bias(X):
    bias = np.ones((X.shape[0], 1))
    return np.concatenate((bias, X), axis=1)

def fit(X, y, lr = 0.01, num_iter = 5):
    
    X = add_bias(X)
    
    theta = np.zeros(X.shape[1])
    
    for _ in range(num_iter):
        z = np.dot(X, theta)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h - y)) / y.size
        theta -= lr * gradient
        theta[-1] = np.sum(h-y)
            
    return theta

def predict(X, theta):
    X = add_bias(X)
    return sigmoid(np.dot(X, theta)).round()

In [10]:
model = fit(X_train, y_train, num_iter = 1)
print(model[:30])

print(loss(model, X_train, y_train))

preds = predict(X_test, model)
(preds == y_test).mean()

[-0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594
 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594
 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594
 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594 -0.00061594
 -0.00061594 -0.00061594 -0.00061594  0.         -0.00061594 -0.00061594]
9.52219601210531


0.5109489051094891

In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print((preds == y_test).mean())

model.intercept_, model.coef_

C:\Users\Edson\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.48905109489051096


(array([-0.00339157]),
 array([[-0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
         -0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
         -0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
         -0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
         -0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
         -0.00339157,  0.        , -0.00339157, -0.00339157, -0.00339157,
         -0.00339157,  0.        , -0.00339157, -0.00339157, -0.00339157,
         -0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
          0.        , -0.00339157, -0.00339157, -0.00339157,  0.        ,
         -0.00339157,  0.        , -0.00339157,  0.        ,  0.        ,
          0.        , -0.00339157, -0.00339157,  0.        ,  0.        ,
         -0.00339157, -0.00339157, -0.00339157, -0.00339157, -0.00339157,
          0.        ,  0.        , -0.00339157,  0.        , -0.00339157,
         -0.003

### Exercício

1. Utilizando a mesma base de dados, encontre o vocabulário apenas na parte de treino e aplique a transformação na parte de teste também. Da forma como foi implementado originalmente, o vocabulário está sendo encontrado em todo o conjunto de dados.

2. Implemente o método de otimização por algoritmo genético e veja se consegue resultados melhores que a regressão logística com os parâmetros default do scikit-learn. Considere no algoritmo genético que o cruzamento será feito de forma uniforme, utilizando um valor aleatório para definir qual gene será herdado de cada pai; também considere que a geração seguinte só deve ser feita de filhos da geração anterior. Inicialize os pesos de forma aleatória utilizando a função própria para isso do numpy e torne fixo o fator de aleatoriedade.

3. Pesquise sobre o conceito de stopwords e aplique-o, analisando se traz melhoras ou pioras para o modelo. Faça a análise considerando a Regressão Logística do scikit-learn e sua versão com algoritmos genéticos.

4. Pesquise sobre o conceito TF-IDF e aplique-o, analisando se traz melhoras ou pioras para o modelo. Também faça a análise considerando utilizando os dois métodos: Regressão Logística do scikit-learn e sua versão com algoritmos genéticos.